In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy.datasets.download import download_url
import shutil
import os
from insitupy import read_xenium
import scanpy as sc

## Previous steps

Download the example data for demonstration: [01_InSituPy_demo_download_data.ipynb](./01_InSituPy_demo_download_data.ipynb).


### Instructions: Setting Up R Integration in Jupyter Notebook

To run sctransform, that calls R in the backend within a Jupyter Notebook, please follow these steps:

---

#### 1. Install R

Download and install R from the [CRAN website](https://cran.r-project.org/):

- **Windows**: Choose the Windows installer and follow the installation prompts.
- **macOS**: Download the macOS package and install it.
- **Linux**: Use your package manager (e.g., `sudo apt install r-base` for Ubuntu/Debian).

---

####  2. Install the `rpy2` Python Package

Open your terminal or command prompt and run:

```bash
pip install rpy2
```

Ensure that you're installing it in the same Python environment that your Jupyter Notebook will use.

---


#### 3. **Add R to the PATH (Windows):**

   - The default path is usually `C:\Program Files\R\R-x.y.z\bin`, where `x.y.z` is the R version number.
   - Press `Win + X` and select **System**.
   - Click on **Advanced system settings**.
   - In the **System Properties** window, click **Environment Variables**.
   - Under **System variables**, scroll down and select **Path**, then click **Edit**.
   - Click **New** and add the path to R's `bin` directory (e.g., `C:\Program Files\R\R-4.1.0\bin`).
   - Click **OK** to close all windows.
   - Restart VisualStudio


In [3]:
# prepare paths
out_dir = Path("demo_dataset") # output directory
data_dir = out_dir / "output-XETG00000__0001879__Replicate 1" # directory of xenium data
image_dir = out_dir / "unregistered_images" # directory of images

In [4]:
xd = read_xenium(data_dir)

In [5]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\Aitana\OneDrive\Documentos\Github\InSituPy\notebooks\demo_dataset\output-XETG00000__0001879__Replicate 1
Metadata file:	experiment.xenium

In [6]:
# read all data modalities at once
xd.load_all()

# alternatively, it is also possible to read each modality separately
# xd.load_cells()
# xd.load_images()
# xd.load_transcripts()
# xd.read_annotations()

Loading annotations...
No `annotations` modality found.
Loading cells...
Loading images...
Loading regions...
No `regions` modality found.
Loading transcripts...


In [7]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\Aitana\OneDrive\Documentos\Github\InSituPy\notebooks\demo_dataset\output-XETG00000__0001879__Replicate 1
Metadata file:	experiment.xenium
    ➤ images
       nuclei:	(25778, 35416)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               nuclear
               cellular
    ➤ transcripts
       DataFrame with shape 42638083 x 7

Filtering 

In [8]:
sc.pp.filter_cells(xd.cells.matrix, min_genes=10)

#### Applying sctransform method to the insitudata object

In [9]:
xd.sctransform()

Applying SCTransform to the main modality (cells.matrix)...

    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Running SCTransform on assay: RNA

R[write to console]: Running SCTransform on layer: counts

R[write to console]: vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

R[write to console]: Variance stabilizing transformation of count matrix of size 313 by 163565

R[write to console]: Model formula is y ~ log_umi

R[write to console]: Get Negative Binomial regression parameters per gene

R[write to console]: Using 313 genes, 5000 cells

R[write to console]: Found 2 outliers - those will be ignored in fitting/regularization step


R[write to console]: Second step: Get residuals using fitted parameters for 313 genes

R[write to console]: Computing corrected count matrix for 313 genes

R[write to console]: Calculating gene attributes

R[write to console]: Wall clock passed: Time difference of 37.93162 secs

R[write to console]: Determine variable features

R[write to console]: Centering data matrix

  |                                        

No alternative modalities found.
SCTransform completed for all modalities.


If we now access the anndata object inside cells.matrix, SCT_adata contains the sctransform counts, while layer counts contains the raw data. 

In [10]:
xd.cells.matrix

AnnData object with n_obs × n_vars = 163565 × 313
    obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'
    varm: 'binned_expression'
    layers: 'SCT_adata', 'counts'